In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import json
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [4]:
class DailyRerankingSystem:
    def __init__(self, model_path, mappings_path):
        """
        Sistema de reranking optimizado para RTX 3050 Ti
        """
        # Cargar modelo con configuración GPU
        physical_devices = tf.config.experimental.list_physical_devices('GPU')
        if len(physical_devices) > 0:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
        
        self.model = tf.keras.models.load_model(model_path)
        
        with open(mappings_path, 'rb') as f:
            mappings = pickle.load(f)
            self.user_to_idx = mappings['user_to_idx']
            self.item_to_idx = mappings['item_to_idx']
            self.city_to_idx = mappings['city_to_idx']
        
        self.weather_cache = {}
        self.trend_cache = {}
        
        print(f"Sistema de reranking cargado con {len(self.user_to_idx)} usuarios y {len(self.item_to_idx)} items")
        
    def get_real_time_weather_fast(self, city):
        """
        Obtención rápida de clima (versión simplificada)
        """
        if city in self.weather_cache:
            cache_time = self.weather_cache[city]['timestamp']
            if (datetime.now() - cache_time).seconds < 7200:  # Cache por 2 horas
                return self.weather_cache[city]['data']
        
        # Datos simulados para velocidad (en producción usar API real)
        weather_data = {
            'temperature': np.random.uniform(15, 30),
            'humidity': np.random.uniform(40, 80),
            'wind_speed': np.random.uniform(0, 15),
            'precipitation': np.random.uniform(0, 5),
            'description': np.random.choice(['soleado', 'nublado', 'lluvia ligera']),
            'conditions': np.random.choice(['Clear', 'Clouds', 'Rain'])
        }
        
        self.weather_cache[city] = {
            'data': weather_data,
            'timestamp': datetime.now()
        }
        
        return weather_data
    
    def get_trending_activities_fast(self, city, time_window_days=3):
        """
        Detección rápida de trends (simplificada)
        """
        cache_key = f"{city}_{time_window_days}"
        
        if cache_key in self.trend_cache:
            cache_time = self.trend_cache[cache_key]['timestamp']
            if (datetime.now() - cache_time).seconds < 3600:  # Cache por 1 hora
                return self.trend_cache[cache_key]['data']
        
        # Simulación rápida de trends
        trending_data = {
            'trending_items': list(np.random.choice(list(self.item_to_idx.keys()), size=10, replace=False)),
            'viral_sentiment': np.random.uniform(0.6, 0.9),
            'activity_surge': np.random.uniform(1.0, 1.5),
            'peak_hours': [10, 14, 18]
        }
        
        self.trend_cache[cache_key] = {
            'data': trending_data,
            'timestamp': datetime.now()
        }
        
        return trending_data
    
    def calculate_dynamic_factors_fast(self, city, current_time=None):
        """
        Cálculo rápido de factores dinámicos
        """
        if current_time is None:
            current_time = datetime.now()
        
        # Obtener datos básicos
        weather_data = self.get_real_time_weather_fast(city)
        trending_data = self.get_trending_activities_fast(city)
        
        # Factores simplificados para velocidad
        weather_factor = self._calculate_weather_factor_fast(weather_data)
        temporal_factor = self._calculate_temporal_factor_fast(current_time)
        trending_factor = trending_data['activity_surge']
        seasonal_factor = self._calculate_seasonal_factor_fast(current_time)
        
        return {
            'weather': weather_factor,
            'temporal': temporal_factor,
            'trending': trending_factor,
            'seasonal': seasonal_factor,
            'raw_data': {
                'weather': weather_data,
                'trending': trending_data
            }
        }
    
    def _calculate_weather_factor_fast(self, weather_data):
        """
        Cálculo rápido de factor climático
        """
        temp = weather_data['temperature']
        precipitation = weather_data['precipitation']
        conditions = weather_data['conditions']
        
        # Cálculo simplificado
        temp_factor = 1.0 if 18 <= temp <= 25 else max(0.5, 1.0 - abs(temp - 21.5) * 0.02)
        rain_factor = max(0.3, 1.0 - precipitation * 0.15)
        condition_factor = {'Clear': 1.0, 'Clouds': 0.8, 'Rain': 0.4}.get(conditions, 0.7)
        
        overall_factor = (temp_factor * 0.4 + rain_factor * 0.4 + condition_factor * 0.2)
        
        return {
            'overall': overall_factor,
            'temperature': temp_factor,
            'precipitation': rain_factor,
            'conditions': condition_factor,
            'indoor_boost': 1.3 if precipitation > 3 else 1.0,
            'outdoor_penalty': 0.6 if precipitation > 8 else 1.0
        }
    
    def _calculate_temporal_factor_fast(self, current_time):
        """
        Factor temporal simplificado
        """
        hour = current_time.hour
        weekday = current_time.weekday()
        
        # Factores rápidos
        hour_factor = 1.0 if 10 <= hour <= 18 else 0.7
        weekday_factor = 1.2 if weekday >= 5 else 0.9  # Fin de semana boost
        
        activity_preference = 'peak' if 10 <= hour <= 18 else 'off_peak'
        day_type = 'weekend' if weekday >= 5 else 'weekday'
        
        return {
            'hour_factor': hour_factor,
            'weekday_factor': weekday_factor,
            'activity_preference': activity_preference,
            'day_type': day_type,
            'peak_time': 10 <= hour <= 18
        }
    
    def _calculate_seasonal_factor_fast(self, current_time):
        """
        Factor estacional rápido
        """
        month = current_time.month
        
        # Factores simplificados por mes
        seasonal_factors = {
            1: 0.6, 2: 0.6, 3: 0.8, 4: 1.0, 5: 1.2, 6: 1.3,
            7: 1.5, 8: 1.5, 9: 1.2, 10: 1.0, 11: 0.7, 12: 0.8
        }
        
        return {
            'factor': seasonal_factors[month],
            'season': ['winter', 'spring', 'summer', 'autumn'][((month-1)//3)],
            'peak_season': month in [6, 7, 8]
        }
    
    def rerank_recommendations_fast(self, base_recommendations, city, user_preferences=None, max_items=50):
        """
        Reranking rápido optimizado para RTX 3050 Ti
        """
        print(f"Re-rankeando {min(len(base_recommendations), max_items)} recomendaciones para {city}...")
        
        # Limitar número de recomendaciones para velocidad
        limited_recs = base_recommendations[:max_items]
        
        # Obtener factores dinámicos
        dynamic_factors = self.calculate_dynamic_factors_fast(city)
        
        # Reranking vectorizado para velocidad
        reranked_recs = []
        
        for rec in limited_recs:
            base_score = rec['combined_score']
            
            # Aplicar factores de manera simplificada
            weather_adj = dynamic_factors['weather']['overall']
            temporal_adj = dynamic_factors['temporal']['hour_factor'] * dynamic_factors['temporal']['weekday_factor']
            trending_adj = dynamic_factors['trending']
            seasonal_adj = dynamic_factors['seasonal']['factor']
            
            # Score final simplificado
            final_score = base_score * (
                0.4 + weather_adj * 0.2 + temporal_adj * 0.2 + 
                trending_adj * 0.1 + seasonal_adj * 0.1
            )
            
            reranked_rec = rec.copy()
            reranked_rec.update({
                'original_score': base_score,
                'final_score': final_score,
                'weather_boost': weather_adj > 1.0,
                'temporal_boost': temporal_adj > 1.0,
                'trending_boost': trending_adj > 1.2,
                'seasonal_boost': seasonal_adj > 1.0
            })
            
            reranked_recs.append(reranked_rec)
        
        # Ordenar por score final
        reranked_recs.sort(key=lambda x: x['final_score'], reverse=True)
        
        return reranked_recs

In [10]:
class ExplainabilityEngine:
    def __init__(self, reranking_system):
        self.reranking_system = reranking_system
    
    def generate_explanation(self, recommendation, user_context=None):
        """
        Genera explicación para una recomendación
        """
        explanations = []
        
        # Explicación de score base
        base_score = recommendation['original_score']
        explanations.append(
            f"Puntuación base: {base_score:.2f} basada en tus preferencias y actividad histórica"
        )
        
        # Explicaciones de ajustes
        adjustments = recommendation['adjustments']
        dynamic_factors = recommendation['dynamic_factors']
        
        # Clima
        weather_adj = adjustments['weather']
        if weather_adj > 1.1:
            weather_desc = dynamic_factors['raw_data']['weather']['description']
            explanations.append(
                f"Recomendado por el clima actual: {weather_desc} (+{(weather_adj-1)*100:.0f}%)"
            )
        elif weather_adj < 0.9:
            explanations.append(
                f"Ten en cuenta las condiciones climáticas actuales"
            )
        
        # Temporal
        temporal_adj = adjustments['temporal']
        temporal_info = dynamic_factors['temporal']
        if temporal_info['peak_time']:
            explanations.append(
                f"Hora ideal para esta actividad ({temporal_info['activity_preference']})"
            )
        
        if temporal_info['day_type'] == 'weekend':
            explanations.append(
                f"Perfecto para el fin de semana"
            )
        
        # Trending
        trending_adj = adjustments['trending']
        if trending_adj > 1.2:
            explanations.append(
                f"Actividad muy popular últimamente"
            )
        
        # Estacional
        seasonal_adj = adjustments['seasonal']
        seasonal_info = dynamic_factors['seasonal']
        if seasonal_info['peak_season']:
            explanations.append(
                f"Temporada alta - experiencia óptima"
            )
        elif seasonal_adj < 0.8:
            explanations.append(
                f"Temporada baja - posibles descuentos"
            )
        
        return {
            'final_score': recommendation['final_score'],
            'explanations': explanations,
            'confidence': min(1.0, recommendation['final_score']),
            'factors': {
                'weather_impact': weather_adj,
                'time_relevance': temporal_adj,
                'popularity_trend': trending_adj,
                'seasonal_fit': seasonal_adj
            }
        }
    
    def create_explanation_report(self, reranked_recommendations, top_n=5):
        """
        Crea reporte de explicaciones para las top N recomendaciones
        """
        report = {
            'timestamp': datetime.now().isoformat(),
            'total_recommendations': len(reranked_recommendations),
            'top_recommendations': []
        }
        
        for i, rec in enumerate(reranked_recommendations[:top_n]):
            explanation = self.generate_explanation(rec)
            
            rec_report = {
                'rank': i + 1,
                'item_id': rec['item_id'],
                'explanation': explanation,
                'score_breakdown': {
                    'original': rec['original_score'],
                    'final': rec['final_score'],
                    'improvement': rec['final_score'] - rec['original_score']
                }
            }
            
            report['top_recommendations'].append(rec_report)
        
        return report
    
    def visualize_score_factors(self, recommendation):
        """
        Crea visualización de factores de score
        """
        factors = recommendation['adjustments']
        
        # Crear gráfico de barras
        plt.figure(figsize=(10, 6))
        
        factor_names = list(factors.keys())
        factor_values = list(factors.values())
        
        colors = ['skyblue', 'lightgreen', 'orange', 'pink']
        bars = plt.bar(factor_names, factor_values, color=colors)
        
        # Añadir línea base en 1.0
        plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Baseline')
        
        plt.title(f'Factores de Ajuste - Item {recommendation["item_id"]}')
        plt.ylabel('Factor de Ajuste')
        plt.ylim(0, max(factor_values) * 1.1)
        
        # Añadir valores en las barras
        for bar, value in zip(bars, factor_values):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f'{value:.2f}', ha='center', va='bottom')
        
        plt.legend()
        plt.tight_layout()
        plt.show()

In [8]:
class TourismRecommenderSystem:
    def __init__(self, model_path, mappings_path):
        self.reranking_system = DailyRerankingSystem(model_path, mappings_path)
        self.explainability_engine = ExplainabilityEngine(self.reranking_system)
    
    def get_daily_recommendations(self, user_id, city, num_recommendations=10):
        """
        Obtiene recomendaciones diarias completas con explicabilidad
        """
        print(f"Generando recomendaciones diarias para usuario {user_id} en {city}")
        
        # 1. Generar recomendaciones base (desde el modelo de DL)
        # Aquí conectarías con el modelo de la parte anterior
        base_recommendations = self._get_base_recommendations(user_id, city, num_recommendations * 2)
        
        # 2. Aplicar reranking dinámico
        reranked_recommendations = self.reranking_system.rerank_recommendations(
            base_recommendations, city
        )
        
        # 3. Generar explicaciones
        explanation_report = self.explainability_engine.create_explanation_report(
            reranked_recommendations, top_n=num_recommendations
        )
        
        return {
            'recommendations': reranked_recommendations[:num_recommendations],
            'explanations': explanation_report,
            'metadata': {
                'city': city,
                'user_id': user_id,
                'timestamp': datetime.now(),
                'factors_applied': True
            }
        }
    
    def _get_base_recommendations(self, user_id, city, num_recs):
        """
        Placeholder para obtener recomendaciones base del modelo DL
        """
        # Aquí conectarías con el modelo entrenado
        # Por ahora retorno datos simulados
        return [
            {
                'item_id': f'item_{i}',
                'predicted_rating': np.random.uniform(0.6, 0.95),
                'predicted_sentiment': np.random.uniform(0.3, 0.9),
                'interaction_probability': np.random.uniform(0.5, 0.9),
                'combined_score': np.random.uniform(0.6, 0.9)
            }
            for i in range(num_recs)
        ]